## Setting up the environment

In [2]:
!pip install hydra-core==1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
!pip install scipy==1.7.3

BRANCH = 'main'
# If you're using Google Colab and not running locally, uncomment and run this cell.
!apt-get install sox libsndfile1 ffmpeg
!pip install wget unidecode pynini==2.1.4
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

#=== load the repo and data (Thanks Synthbot) ===
!sudo dpkg --configure -a
!sudo apt -qq install -y sox
!git clone "https://github.com/synthbot-anon/synthbot.git" /content/synthbot
!(cd /content/synthbot; git checkout experimental)
!python3 -m pip install pysoundfile 
!python3 -m pip install librosa

!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.2).
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3 sox
0 upgraded, 8 newly installed, 0 to remove and 49 not upgraded.
Need to get 760 kB of archives.
After this operation, 6,717 kB of additional disk space will be used.
Get:1 http://archive

In [3]:
home_path = !(echo $HOME)
home_path = home_path[0]
print(home_path)

/root


In [4]:
#downloading the dataset
id='12ClseQAzkqsqCNMClmptvbSjfs2Ulcei'
character='applejack'
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id={id}' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id={id}" -O {character+'.tar'} && rm -rf /tmp/cookies.txt


--2022-07-27 13:56:24--  https://docs.google.com/uc?export=download&confirm=t&id=12ClseQAzkqsqCNMClmptvbSjfs2Ulcei
Resolving docs.google.com (docs.google.com)... 108.177.121.139, 108.177.121.102, 108.177.121.100, ...
Connecting to docs.google.com (docs.google.com)|108.177.121.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/j5cjnhpd4tlr0m7j4vdpd1odmlt7qeu3/1658930175000/05434657448262868821/*/12ClseQAzkqsqCNMClmptvbSjfs2Ulcei?e=download&uuid=e66954af-b0ee-4d61-b9a7-f6eb69f2b4ca [following]
--2022-07-27 13:56:24--  https://doc-0c-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/j5cjnhpd4tlr0m7j4vdpd1odmlt7qeu3/1658930175000/05434657448262868821/*/12ClseQAzkqsqCNMClmptvbSjfs2Ulcei?e=download&uuid=e66954af-b0ee-4d61-b9a7-f6eb69f2b4ca
Resolving doc-0c-84-docs.googleusercontent.com (doc-0c-84-docs.googleusercontent.com)... 173.194.192.

In [5]:
skip_noisy = True # Disable to train with Noisy Data included
allowed_emotions = """
Anxious
Angry
Annoyed
Amused
Confused
Crazy
Disgust
Exhausted
Fear
Happy
Neutral
Sad
Serious
Singing
Shouting
Surprised
Smug
Love
Sarcastic
Tired
Whispering
Whining
""".split("\n")[1:-1]

percentage_training_data = 0.95 # 90% of Data will be used for training, 5% for Validation

##############################################################################################

import sys
sys.path.append('/content/synthbot/src')
from ponysynth.corpus import ClipperArchive, phoneme_transcription
import librosa
import subprocess

archive_fn = "/content/applejack.tar"
archive = ClipperArchive(archive_fn)

data_path = 'applejack'
allowed_emotions = [x.lower() for x in allowed_emotions]
!mkdir {data_path}
!mkdir {data_path+"/out"}
all_clips = []; all_clips_arpa = []; skipped_count=0; too_short_count=0; emotion_skip=0
for key in archive.keys(): # write the audio files for processing in bash terminal
  audio = archive.read_audio(key)
  audio_fn = '{}/{}.wav'.format(data_path, key)
  audio_fn_ = '{}/{}.wav'.format(data_path+"/out", key)
  with open(audio_fn, 'wb') as audio_out:
    audio_out.write(audio.read())

/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)


In [6]:
%%script bash
# trim all 48Khz files
cd applejack
for input in *.wav; do
  output="out/$input"
  sox "$input" "$output" silence 1 0.05 0.1% reverse silence 1 0.05 0.1% reverse;
done

In [7]:
import os

should_continue = 0 # should run "continue" command inside outer loop.
for key in archive.keys():
  label = archive.read_label(key)
  audio_fn = '{}/{}.wav'.format(data_path, key)
  audio_fn_ = '{}/{}.wav'.format(data_path+"/out", key)
  if (label['noise'] in ['Very Noisy','Noisy']) and skip_noisy: os.remove(audio_fn_); skipped_count+=1; continue

  for tag in label['tags']:
     if tag.lower() not in allowed_emotions:
       try: os.remove(audio_fn_)
       except: pass
       print(tag+" emotion not in list"); emotion_skip+=1; should_continue = 1; break # this is supposed to break the outer loop
  if should_continue: should_continue = 0; continue

  audio = archive.read_audio(key)
  transcript = label['utterance']['content']
  
  if os.stat(audio_fn_).st_size < 71602:
    #print("Skipping Audio, Duration: "+str(len(librosa.core.load(audio_fn_, sr=48000)[0])/48000))
    try: os.remove(audio_fn_)
    except: pass
    too_short_count+=1; continue # Skips files based on size.

  filelist_line = "{}|{}".format(audio_fn_, transcript)
  all_clips.append(filelist_line)
  filelist_line = "{}|{}".format(audio_fn_, phoneme_transcription(label))
  all_clips_arpa.append(filelist_line)
    
print(str(skipped_count)+" Files are too Noisy.")
print(str(emotion_skip)+" Files contain an emotion not in permitted emotions")
print(str(too_short_count)+" Files are too short")
print(str(len(list(archive.keys()))-(skipped_count+too_short_count+emotion_skip))+" Files kept in dataset.")


####################################################################################################################
# shuffle the training data
import random
random.seed(0)
random.shuffle(all_clips)

# get train, test, validation splits
num_clips = len(all_clips)
train_end = int(num_clips * percentage_training_data)

train = all_clips[:train_end]
validation = all_clips[train_end:]

train_arpa = all_clips_arpa[:train_end]
validation_arpa = all_clips_arpa[train_end:]

# dump the info to filelist files
with open('train_filelist.txt', 'w') as train_out:
  train_out.write('\n'.join(train)+'\n'+'\n'.join(train_arpa)+"")
with open('val_filelist.txt', 'w') as val_out:
  val_out.write('\n'.join(validation)+'\n'+'\n'.join(validation_arpa)+"")

##################################################################################################################

#imports
import os
import json
import random
import librosa

import torch
import IPython.display as ipd
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
from pathlib import Path

from nemo.collections.tts.models import FastPitchModel

######################################################################################

#creating the json files for training and validation
with open('/content/train_filelist.txt') as f :
  records = f.readlines()

print("Number of records : ",len(records))

train_manifest = 'fastpitch_train.json'

train_rec = []
random.shuffle(records)
count = 0

for i in records :
  #if count > 1200 :
  #  break
  
  i = i.split('|')
  audio_filepath = i[0]
  text = i[-1].strip('\n')
  if '{' in text :
    #print(text)
    continue
  count = count + 1

  duration = librosa.get_duration(filename=audio_filepath)
  r = {
       "audio_filepath" : audio_filepath,
       "text" : text,
       "duration" : round(duration,1),
       "text_no_preprocessing" : text
    }

  train_rec.append(r)

with open(train_manifest, "w") as f:
    for s in train_rec:
        f.write(json.dumps(s) + '\n')
        
print("Training Data : ", len(train_rec))

##########################################################################################

with open('/content/val_filelist.txt') as f :
  records_val = f.readlines()

print("Number of records : ", len(records_val))

val_manifest = 'fastpitch_val.json'
count = 0
val_rec = []
random.shuffle(records_val)

for i in records_val:
  #if count > 50 :
  #  break
    
  i = i.split('|')
  audio_filepath = i[0]
  text = i[-1].strip('\n')

  if '{' in text :
    #print(text)
    continue
  count = count + 1
  duration = librosa.get_duration(filename=audio_filepath)
  r = {
       "audio_filepath" : audio_filepath,
       "text" : text,
       "duration" : round(duration,1),
       "text_no_preprocessing" : text
    }

  val_rec.append(r)

print(len(val_rec))
with open(val_manifest, "w") as f:
    for s in val_rec:
        f.write(json.dumps(s) + '\n')

####################################################################################

FastPitchModel.from_pretrained("tts_en_fastpitch")
nemo_files = [p for p in Path(f"{home_path}/.cache/torch/NeMo/").glob("**/tts_en_fastpitch_align.nemo")]
print(f"Copying {nemo_files[0]} to ./")
Path("./tts_en_fastpitch_align.nemo").write_bytes(nemo_files[0].read_bytes())

###################################################################################

!wget https://raw.githubusercontent.com/nvidia/NeMo/main/examples/tts/fastpitch_finetune.py

!mkdir -p conf \
&& cd conf \
&& wget https://raw.githubusercontent.com/nvidia/NeMo/main/examples/tts/conf/fastpitch_align_v1.05.yaml \
&& cd ..

#######################################
# additional files
!mkdir -p tts_dataset_files && cd tts_dataset_files \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tts_dataset_files/cmudict-0.7b_nv22.07 \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tts_dataset_files/heteronyms-030921 \
&& wget wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/nemo_text_processing/text_normalization/en/data/whitelist/lj_speech.tsv \
&& cd ..



2768 Files are too Noisy.
0 Files contain an emotion not in permitted emotions
203 Files are too short
2486 Files kept in dataset.


[NeMo W 2022-07-27 13:57:21 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-07-27 13:57:21 experimental:28] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.


Number of records :  4722
Training Data :  2361
Number of records :  250
125
[NeMo I 2022-07-27 13:57:22 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/tts_en_fastpitch/versions/1.8.1/files/tts_en_fastpitch_align.nemo to /root/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_en_fastpitch_align/26d7e09971f1d611e24df90c7a9d9b38/tts_en_fastpitch_align.nemo
[NeMo I 2022-07-27 13:57:28 common:789] Instantiating model from pre-trained checkpoint
[NeMo I 2022-07-27 13:57:31 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-27 13:57:51 experimental:28] Module <class 'nemo.collections.tts.torch.g2ps.IPAG2P'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-07-27 13:57:51 g2ps:87] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-27 13:57:51 modelPT:143] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /ws/LJSpeech/nvidia_ljspeech_train_clean_ngc.json
      sample_rate: 22050
      sup_data_path: /raid/LJSpeech/supplementary
      sup_data_types:
      - align_prior_matri

[NeMo I 2022-07-27 13:57:52 features:217] PADDING: 1
[NeMo I 2022-07-27 13:57:56 save_restore_connector:243] Model FastPitchModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_en_fastpitch_align/26d7e09971f1d611e24df90c7a9d9b38/tts_en_fastpitch_align.nemo.
Copying /root/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_en_fastpitch_align/26d7e09971f1d611e24df90c7a9d9b38/tts_en_fastpitch_align.nemo to ./
--2022-07-27 13:57:57--  https://raw.githubusercontent.com/nvidia/NeMo/main/examples/tts/fastpitch_finetune.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1751 (1.7K) [text/plain]
Saving to: ‘fastpitch_finetune.py’

fastpitch_finetune. 100%[===================>]   1.71K  --.-KB/s    in 0s      

2022-07-27 13:57:57 (36.7 MB/s) - ‘fastpitch_

In [9]:
from nemo.collections.tts.torch.g2ps import EnglishG2p
from nemo.collections.tts.torch.data import TTSDataset
from nemo_text_processing.text_normalization.normalize import Normalizer
from nemo.collections.tts.torch.tts_tokenizers import EnglishPhonemesTokenizer, EnglishCharsTokenizer

import torch
from tqdm.notebook import tqdm


# Text normalizer
text_normalizer = Normalizer(
    lang="en", 
    input_case="cased", 
    whitelist="./tts_dataset_files/lj_speech.tsv"
)

text_normalizer_call_kwargs = {
    "punct_pre_process": True,
    "punct_post_process": True
}

# Text tokenizer
text_tokenizer = EnglishCharsTokenizer()

def pre_calculate_supplementary_data(sup_data_path, sup_data_types, text_tokenizer, text_normalizer, text_normalizer_call_kwargs):
    # init train and val dataloaders
    stages = ["train", "val"]
    stage2dl = {}
    for stage in stages:
        ds = TTSDataset(
            manifest_filepath=f"fastpitch_{stage}.json",
            sample_rate=16000,
            sup_data_path=sup_data_path,
            sup_data_types=sup_data_types,
            n_fft=1024,
            win_length=1024,
            hop_length=256,
            window="hann",
            n_mels=80,
            lowfreq=0,
            highfreq=8000,
            text_tokenizer=text_tokenizer,
            text_normalizer=text_normalizer,
            text_normalizer_call_kwargs=text_normalizer_call_kwargs

        ) 
        stage2dl[stage] = torch.utils.data.DataLoader(ds, batch_size=1, collate_fn=ds._collate_fn, num_workers=0)

    # iteration over dataloaders
    pitch_mean, pitch_std, pitch_min, pitch_max = None, None, None, None
    for stage, dl in stage2dl.items():
        pitch_list = []
        print(stage)
        print(dl)
        for batch in tqdm(dl, total=len(dl)):
            tokens, tokens_lengths, audios, audio_lengths, attn_prior, pitches, pitches_lengths = batch
            pitch = pitches.squeeze(0)
            pitch_list.append(pitch[pitch != 0])

        if stage == "train":
            pitch_tensor = torch.cat(pitch_list)
            pitch_mean, pitch_std = pitch_tensor.mean().item(), pitch_tensor.std().item()
            pitch_min, pitch_max = pitch_tensor.min().item(), pitch_tensor.max().item()
    
    print("Pitch_mean : ",pitch_mean)
    print("Pitch_std : ",pitch_std)
    print("Pitch_min : ",pitch_min)
    print("Pitch_max : ",pitch_max)
            
    return pitch_mean, pitch_std, pitch_min, pitch_max

fastpitch_sup_data_path = "fastpitch_sup_data_folder"
sup_data_types = ["align_prior_matrix", "pitch"]

pitch_mean, pitch_std, pitch_min, pitch_max = pre_calculate_supplementary_data(
    fastpitch_sup_data_path, sup_data_types, text_tokenizer, text_normalizer, text_normalizer_call_kwargs
)

[NeMo I 2022-07-27 10:41:54 tokenize_and_classify:87] Creating ClassifyFst grammars.
[NeMo I 2022-07-27 10:42:20 data:197] Loading dataset from fastpitch_train.json.


2361it [00:30, 76.43it/s]

[NeMo I 2022-07-27 10:42:51 data:232] Loaded dataset with 2361 files.


[NeMo I 2022-07-27 10:42:51 data:234] Dataset contains 1.55 hours.
[NeMo I 2022-07-27 10:42:51 data:336] Pruned 0 files. Final dataset contains 2361 files
[NeMo I 2022-07-27 10:42:51 data:339] Pruned 0.00 hours. Final dataset contains 1.55 hours.
[NeMo I 2022-07-27 10:42:51 data:197] Loading dataset from fastpitch_val.json.


125it [00:01, 71.51it/s]

...

  0%|          | 0/2361 [00:00<?, ?it/s]

[NeMo W 2022-07-27 10:42:53 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
      warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
    
[NeMo W 2022-07-27 10:42:56 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
      warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
    


val


  0%|          | 0/125 [00:00<?, ?it/s]

Pitch_mean :  324.42596435546875
Pitch_std :  112.51094055175781
Pitch_min :  65.4063949584961
Pitch_max :  2093.004638671875


In [9]:
#setting up wandb credentials
#weights and biases 
wandb_api_key = "d06dcb679ec9cf0d71c81cb750d142b6b90f3d25"
wandb_project_name = 'MLP'
wandb_run_name = character

!wandb login ${wandb_api_key}

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [11]:
#resuming the training 
#+init_from_ptl_ckpt=Fastpitch_epoch_356.ckpt
# trainer.max_epochs=500 ~trainer.max_epochs 
#+init_from_nemo_model=tts_en_fastpitch_align.nemo \
! (python fastpitch_finetune.py --config-name=fastpitch_align_v1.05.yaml \
  train_dataset=fastpitch_train.json \
  validation_datasets=fastpitch_val.json \
  sup_data_path=fastpitch_sup_data \
  phoneme_dict_path=tts_dataset_files/cmudict-0.7b_nv22.07 \
  heteronyms_path=tts_dataset_files/heteronyms-030921 \
  whitelist_path=tts_dataset_files/lj_speech.tsv \
  exp_manager.exp_dir=training_logs \
  +init_from_nemo_model=tts_en_fastpitch_align.nemo \
  trainer.max_epochs=1000 ~trainer.max_epochs \
  trainer.check_val_every_n_epoch=7 \
  model.train_ds.dataloader_params.batch_size=8 model.validation_ds.dataloader_params.batch_size=8 \
  model.n_speakers=1 model.pitch_mean=324.43 model.pitch_std=112.51 \
  model.pitch_fmin=65.41 model.pitch_fmax=2093 model.optim.lr=2e-4 \
  +exp_manager.create_wandb_logger=true \
  +exp_manager.wandb_logger_kwargs.name='applejack' \
  +exp_manager.wandb_logger_kwargs.project='MLP' \
  ~model.optim.sched model.optim.name=adam trainer.devices=1 trainer.strategy=null \
  +model.text_tokenizer.add_blank_at=true \
)

[NeMo W 2022-07-27 14:00:55 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-07-27 14:00:56 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
      _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)
    
[NeMo W 2022-07-27 14:01:05 experimental:28] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
Using 16bit native Automatic Mixed Precision (AMP)
[NeMo W 2022-07-27 14:01:05 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:95: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_

## Checking the checkpoint

In [13]:
#imports
import os
import json
import librosa

import torch
import IPython.display as ipd
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
from pathlib import Path

from nemo.collections.tts.models import FastPitchModel
from nemo.collections.tts.models import HifiGanModel
from nemo.collections.tts.torch.helpers import BetaBinomialInterpolator
import soundfile as sf
import numpy as np

#helper functions
def infer(spec_gen_model, vocoder_model, str_input, speaker=None):
    """
    Synthesizes spectrogram and audio from a text string given a spectrogram synthesis and vocoder model.
    
    Args:
        spec_gen_model: Spectrogram generator model (FastPitch in our case)
        vocoder_model: Vocoder model (HiFiGAN in our case)
        str_input: Text input for the synthesis
        speaker: Speaker ID
    
    Returns:
        spectrogram and waveform of the synthesized audio.
    """
    with torch.no_grad():
        parsed = spec_gen_model.parse(str_input)
        if speaker is not None:
            speaker = torch.tensor([speaker]).long().to(device=spec_gen_model.device)
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, speaker=speaker)
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
        
    if spectrogram is not None:
        if isinstance(spectrogram, torch.Tensor):
            spectrogram = spectrogram.to('cpu').numpy()
        if len(spectrogram.shape) == 3:
            spectrogram = spectrogram[0]
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return spectrogram, audio

def custom_infer(transcript, spec_model, vocoder, download=None):
  spec, audio = infer(spec_model, vocoder, transcript)
  print("\n\n\n\n")
  ipd.display(ipd.Audio(audio, rate=22050))
  %matplotlib inline
  plt.show()

def try_infering_with_finetuned_fastpitch(checkpointpath, sentence):
  spec_model = FastPitchModel.load_from_checkpoint(checkpointpath)
  spec_model.eval().cuda()

  vocoder = HifiGanModel.from_pretrained("tts_hifigan")
  vocoder = vocoder.eval().cuda()

  custom_infer(sentence, spec_model, vocoder)

In [24]:
checkpoint = '/content/drive/MyDrive/TTS_finetuning/apple_jack/FastPitch--v_loss=1.1907-epoch=132-last.ckpt'

In [25]:
spec_model = FastPitchModel.load_from_checkpoint(checkpoint)
spec_model.eval().cuda()

vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

[NeMo I 2022-07-28 06:00:17 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-28 06:00:40 g2ps:87] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-28 06:00:40 modelPT:143] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: fastpitch_train.json
      sample_rate: 22050
      sup_data_path: fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min_duration: 0.1
      ignore_file: null
      

[NeMo I 2022-07-28 06:00:40 features:217] PADDING: 1
[NeMo I 2022-07-28 06:00:40 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2022-07-28 06:00:40 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-07-28 06:00:40 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-28 06:00:43 modelPT:143] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-07-28 06:00:43 modelPT:150] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-07-28 06:00:43 features:217] PADDING: 0


[NeMo W 2022-07-28 06:00:43 features:195] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-07-28 06:00:43 features:217] PADDING: 0
[NeMo I 2022-07-28 06:00:44 save_restore_connector:243] Model HifiGanModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.


In [26]:
custom_infer(" I am a dragon and I like to fly over the town. ", spec_model, vocoder)

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
!cp -r /content/training_logs/FastPitch/2022-07-27_14-01-05/checkpoints -t /content/drive/MyDrive/TTS_finetuning/apple_jack/